In [1]:
import requests
import pandas as pd

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

In [2]:
!pip install beautifulsoup4

In [3]:
!pip install lxml

In [4]:
!pip install html5lib


In [5]:
from bs4 import BeautifulSoup

Weblink 

In [6]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Scrape the website and read the table from it

In [7]:
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")
results = soup.find('table', attrs={'class': 'wikitable sortable'})

rows = results.find_all('tr')
data = []

cols = rows[0].find_all('th')
headers = [ele.text.strip() for ele in cols]

for row in rows[1:]:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])

result = pd.DataFrame(data, columns=headers)
result = result[result['Borough']!="Not assigned"].reset_index(drop=True)
result

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


In [8]:
result.shape

(210, 3)

In [9]:

#geolocator = Nominatim(user_agent="Toronto_explorer")
#for code in result['Postcode']:
#    longlat = None
#    while (longlat is None):
#        print(code)
#        longlat = geolocator.geocode('{}, Toronto, ontario'.format(code))
#    lat.append(longlat.latitude)   
#    long.append(longlat.longitude)
#    print(lat)


Get the longitude and latitude from the geospatial coordinates file

In [10]:
geo_df = pd.read_csv("Geospatial_Coordinates.csv")
longlat_dict = {}
for code, lt, ln in zip(geo_df['Postal Code'], geo_df['Latitude'], geo_df['Longitude']):
     longlat_dict[code] = [lt,ln]

Create columns longitude and latitude from geospatial data frame 

In [11]:
lat=[]
long=[]
for code in result['Postcode']:
    lonlat = longlat_dict[code]
    lat.append(lonlat[0])
    long.append(lonlat[1])

Add the location details to result data frame

In [12]:
result['Latitude'] = lat
result['Longitude'] = long
result

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
...,...,...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West,43.628841,-79.520999
206,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
207,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
208,M8Z,Etobicoke,Royal York South West,43.628841,-79.520999
